# Idiomy - wydajność

## Konkatenacja ciągów

W przypadku łączenia mniejszych stringów w dłuższy ciąg znaków, należy unikać poniższej implementacji:

In [3]:
list = [str(i) for i in range(100000)]

In [8]:
%%timeit

joined = ""
for substring in list:
    joined += substring

12.8 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Konkatenacja napisu na końcu innego napisu (``s += substring``) jest bardzo kosztowna obliczeniowo.
Wykonywanie tej operacji w pętli może znacząco spowolnić program.

Dlatego lepiej jest użyć metody ``join``:

In [9]:
%%timeit

joined = ''.join(list)

3.23 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Jako argument można przekazać także wyrażenie listowe, co jest przydatne, gdy napisy są generowane.


In [18]:
lines = ["one", "two", "#comment", "three", "#comment", "four"]

no_comments = [line.capitalize() for line in lines if not line.startswith("#")]
joined = "\n".join(no_comments)

joined

'One\nTwo\nThree\nFour'

Podobnie, zamiast ręcznego łączenia stringów przy pomocy operatora ``+``:

In [20]:
head = "www.site.pl"
query = "&q=costam"
tail = "&u=ktos"
url = "http://" + head + query + tail + "/"

lepiej jest użyć wbudowanej funkcji ``format``:

In [22]:
url = "http://{}{}{}/".format(head, query, tail)

'http://www.site.pl&q=costam&u=ktos/'

lub formatowanego string'a:

In [24]:
url = f"http://{head}{query}{tail}/"
url

'http://www.site.pl&q=costam&u=ktos/'

## Sortowanie

Sortowanie list jest wydajne w Pythonie, niestety użycie funkcji porównującej znacznie spowalnia proces.
Aby tego uniknąć, można zastosować tzw. *"Schwartzian Transform"*, czyli utworzenie listy krotek, której pierwszy element to klucz sortowania:


In [28]:
somelist = [('John', 'Smith', 21), ('Alice', 'Johnson', 24), ('John', 'Doe', 55)]

n = 1  # sortowanie po drugim polu krotki, tj. po nazwisku
nlist = [(x[n], x) for x in somelist]
nlist

[('Smith', ('John', 'Smith', 21)),
 ('Johnson', ('Alice', 'Johnson', 24)),
 ('Doe', ('John', 'Doe', 55))]

In [29]:
nlist.sort()

somelist = [val for (key, val) in nlist]
somelist

[('John', 'Doe', 55), ('Alice', 'Johnson', 24), ('John', 'Smith', 21)]

Krotki są porównywalne, a zatem można je także sortować.
Z dwóch krotek ta jest mniejsza, której pierwszy element jest mniejszy:

In [30]:
(2, 42) < (3, 42)

True

Jeżeli pierwszy element jest taki sam, wówczas porównywany jest drugi element:

In [31]:
('to samo', 2) < ('to samo', 20)

True

Jeżeli drugi element również jest taki sam, wówczas porównywany jest trzeci element itd.

Od Pythona 2.4 i w 3.x możemy podać "klucz" według którego nastąpi sortowanie:

In [33]:
import operator

nlist.sort(key=operator.itemgetter(n))

In [32]:
from random import randint
nlist = [('bla', randint(0, 10000)) for _ in range(10000)]
import operator
%timeit sorted(nlist, key=operator.itemgetter(1))
%timeit sorted([(x[1], x) for x in nlist])

4.73 ms ± 743 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.3 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pętle

Zobaczmy, jak wygląda przykładowa pętla pracująca na liście słów:

In [43]:
import re

words = re.findall('\w+', open('holmes.txt').read().lower())

109214

In [44]:
%%timeit

new_list = []
for word in words:
    new_list.append(word.upper())

36 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


W takim przypadku należy pamiętać, że pętle to nie tylko *for*.

Można użyć wbudowanej funkcji ``map``.

In [45]:
%%timeit

new_list = map(str.upper, words)

367 ns ± 55.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Należy jednak unikać używania funkcji ``map`` i ``filter``, ponieważ wymaga to wywołania funkcji dla każdego elementu kolekcji, co jest dosyć kosztowne.

Dlatego lepiej jest użyć wyrażeń listowych:


In [46]:
%%timeit

new_list = [w.upper() for w in words]

30.7 ms ± 7.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


lub generatorowych:

In [48]:
%%timeit

new_iterator = (w.upper() for w in words)

494 ns ± 36.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Unikanie przeszukiwania

Jeśli nie możemy zrezygnować z pętli *for* należy ograniczyć użycie operatora ``.``.

Z drugiej strony, należy pamiętać, iż zaciemnia to kod, dlatego ta technika powinna być stosowana tylko w sytuacji, gdy program wykonuje się za wolno.
Nie ma sensu stosowanie jej "na wszelki wypadek".

In [50]:
upper = str.upper
new_list = []

append = new_list.append
for word in words:
    append(upper(word))

In [53]:
%%timeit

new_list = []
for word in words:
    new_list.append(word.upper())

34.3 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
%%timeit

upper = str.upper
new_list = []
append = new_list.append

for word in words:
    append(upper(word))

30.9 ms ± 16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Przykład ilustrujący czas wykonania poszczególnych operacji:

### Lokalne vs. globalne

Czas dostępu do zmiennych lokalnych jest o wiele krótszy. Dlatego poniższy kod:

In [55]:
def upper_words(old_list):
    upper = str.upper
    new_list = []
    append = new_list.append
    for word in old_list:
        append(upper(word))
    return newlist

jest szybszy niż:

In [56]:
upper = str.upper

def upper_words(old_list):
    new_list = []
    append = new_list.append
    for word in old_list:
        append(upper(word))
    return new_list

## Słowniki


Częstym motywem podczas pracy ze słownikami jest konieczność sprawdzenia, czy dany klucz istnieje, zanim powiązana wartość zostanie zmieniona.
Na przykład, budowanie słownika częstotliwości występowania słów wygląda następująco:

In [ ]:
import re

words = re.findall('\w+', open('holmes.txt').read().lower())

In [59]:
%%timeit

word_counter = {}
for word in words:
    if word not in word_counter:
        word_counter[word] = 0
    word_counter[word] += 1

31.1 ms ± 7.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Jeżeli słowo występuje w słowniku, to taki kod przeszukuje słownik dwukrotnie - raz w *if* i na końcu pętli, podczas inkrementacji wartości.
Aby tego uniknąć, można użyć wyjątków:


In [60]:
%%timeit

word_counter = {}
for word in words:
    try:
        word_counter[word] += 1
    except KeyError:
        word_counter[word] = 1

28.7 ms ± 8.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Alternatywnie, można też użyć metody ``get``:

In [63]:
%%timeit

word_counter = {}
get = word_counter.get

for word in words:
    word_counter[word] = get(word, 0) + 1

27.1 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Jednak w powyższym przypadku najlepiej będzie użyć ``defaultdict``:

In [65]:
%%timeit

from collections import defaultdict

word_counter = defaultdict(int)
for word in words:
    word_counter[word] += 1



39.5 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Agregacja danych

Python ma dość duży narzut na wywołanie funkcji i należy tego unikać, zwłaszcza w pętlach:

In [2]:
counter = 0

def count(i):
    global counter
    counter = counter + 1

l = list(range(100000))

%timeit for i in l: count(i)

24.8 ms ± 4.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Znacznie szybsze będzie jednokrotne wywołanie funkcji z przekazaniem całej kolekcji elementów:

In [3]:
counter = 0

def fast_count(list):
    global counter
    for i in list:
        counter = counter + 1

%timeit fast_count(l)

11.1 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Sloty

Jeżeli program wykorzystuje dużo pamięci RAM i wynika to z tworzenia milionów małych obiektów, wtedy warto rozważyć dodanie do ich klasy atrybutu ``__slots__``.
Domyślnie, każdy obiekt posiada słownik swoich atrybutów, co generuje spory narzut na pamięć.
Jeśli nasza klasa nie korzysta z dynamicznej natury takiego słownika (tzn. nie dodaje dynamicznie nowych atrybutów), to można wyliczyć wszystkie atrybuty w ``__slots__``.
Obiekty takiej klasy będą zachowywać się podobnie do struktury z C.

In [4]:
class Point(object):
    __slots__ = ["x", "y"]


In [5]:
p = Point()
p.x = 10
p.y = 20
p.z = 30

AttributeError: 'Point' object has no attribute 'z'